**MLP FROM SCRATCH**

This project implements a multi-layer perceptron (MLP) from scratch using numpy, and applies it to classify handwritten digits from the MNIST dataset. The network consists of one hidden layer with ReLU activation and an output layer with softmax activation, because we have 10 outcomes. The notebook covers the forward and backward (2 steps) propagation steps, gradient computation, loss calculation (cross entropy), and parameter updates using stochastic gradient descent.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

In [2]:
def relu(x):
    """
    Apply the ReLU activation function.

    Args:
    x -- NumPy array

    Returns:
    NumPy array with ReLU applied element-wise.
    """
    return np.maximum(0, x)

def der_relu(x):
    """
    Compute the derivative of the ReLU function.

    Args:
    x -- NumPy array

    Returns:
    NumPy array with the derivative of ReLU applied element-wise.
    """
    return np.where(x > 0, 1, 0)

def softmax(x):
    """
    Apply the softmax function to a 2D array.

    Args:
    x -- 2D NumPy array (batch, features)

    Returns:
    NumPy array with softmax applied along the rows.
    """
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

In [3]:
# In forward pass, we need to store output of each layer of NN
# Please note that we need to start from first layer

def forward(X, W0, b0, W1, b1, a=relu):

    # f_i represents the pre-activations at the k-th hidden layer
    # h_i contains the activations at the k-th hidden layer

    # Output of first layer
    f0 = b0 + np.dot(X, W0)
    # Activation of first layer
    h1 = a(f0)

    # Final output activated
    f1 = b1 + np.dot(h1, W1)
    h2 = softmax(f1)

    store = (f0, h1, f1, h2)
    return h2, store

First, we need to compute derivate of input of our softmax function. Since the steps are a bit delicate, let's write the reasoning that justifies the derivative with respect to the input of the softmax function.

$$
\frac{\partial L}{\partial o_i} = -\sum_k y_k \frac{\partial \log p_k}{\partial o_i}
$$

$$
= -\sum_k \frac{y_k}{p_k} \frac{\partial p_k}{\partial o_i}
$$

$$
= -y_i (1 - p_i) + \sum_{k \neq i} y_k p_i
$$

$$
= p_i - y_i
$$

where$$o_i$$is input of softmax.

Since all the details are provided in *Understanding Deep Learning, Prince,* let us rewrite the steps for calculating the derivatives of weights and biases.

$$
\frac{\partial \ell_i}{\partial \beta_k} = \frac{\partial f_k}{\partial \beta_k} \frac{\partial \ell_i}{\partial f_k}
$$

$$
= \frac{\partial}{\partial \beta_k} (\beta_k + \Omega_k h_k) \frac{\partial \ell_i}{\partial f_k}
$$

$$
= \frac{\partial \ell_i}{\partial f_k},
$$

---

$$
\frac{\partial \ell_i}{\partial \Omega_k} = \frac{\partial f_k}{\partial \Omega_k} \frac{\partial \ell_i}{\partial f_k}
$$

$$
= \frac{\partial}{\partial \Omega_k} (\beta_k + \Omega_k h_k) \frac{\partial \ell_i}{\partial f_k}
$$

$$
= \frac{\partial \ell_i}{\partial f_k} h_k^\top.
$$

where:
- $\beta_k$ is bias vector in $k$-th layer.
- $\Omega_k$ is weights matrix $k$-th layer.

Please note that we have avoided calculating the derivatives of the intermediate levels (details on book).


In [4]:
# This is the first step in backward
# Here, we compute derivates of other values, not parameters
def backward_1(X, y, forward_output, W2):

    f0, h1, f1, h2 = forward_output
    m = y.shape[0]

    # Gradient of input of softmax
    df1 = h2 - y

    dh1 = np.dot(df1, W2.T)
    df0 = dh1 * der_relu(f0)

    return df0, df1

# This is the second step in backward
# Here, we compute derivates of parameters
# We prefer normalize the gradient (avoid gradients too unbalanced)
def backward_2(X, y, forward_output, W2, df0, df1):
    f0, h1, f1, h2 = forward_output
    m = y.shape[0]

    dW2 = np.dot(h1.T, df1) / m
    db2 = np.sum(df1, axis=0, keepdims=True) / m

    dW1 = np.dot(X.T, df0) / m
    db1 = np.sum(df0, axis=0, keepdims=True) / m

    return dW1, db1, dW2, db2

def backward(X, y, forward_output, W2):

    # Please note that these are the only derivates we need
    df0, df1 = backward_1(X, y, forward_output, W2)
    dW1, db1, dW2, db2 = backward_2(X, y, forward_output, W2, df0, df1)

    return dW1, db1, dW2, db2

In [5]:
def load_data():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = X_train.reshape(-1, 28*28) / 255.0
    X_test = X_test.reshape(-1, 28*28) / 255.0
    one_hot = OneHotEncoder(sparse_output=False)
    y_train = one_hot.fit_transform(y_train.reshape(-1, 1))
    y_test = one_hot.transform(y_test.reshape(-1, 1))
    return X_train, y_train, X_test, y_test

In [6]:
def init(input_size, hidden_size, output_size):
    W1 = np.random.randn(input_size, hidden_size) * 0.01
    b1 = np.zeros((1, hidden_size))
    W2 = np.random.randn(hidden_size, output_size) * 0.01
    b2 = np.zeros((1, output_size))
    return W1, b1, W2, b2

In [7]:
def loss(y_true, y_pred): # Cross entropy
    offset = 1e-16
    global_loss = np.sum(-np.sum(y_true * np.log(y_pred + offset), axis=1))
    mean_loss = np.mean(-np.sum(y_true * np.log(y_pred + offset), axis=1))
    return global_loss, mean_loss

In [8]:
def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

In [9]:
def train(X, y, hidden_size, output_size, epochs, learning_rate, batch_size, epoch_decay_interval, decay_factor=0.05):
    input_size = X.shape[1]
    W1, b1, W2, b2 = init(input_size, hidden_size, output_size)

    for epoch in range(epochs):

        if epoch % epoch_decay_interval == 0:
            learning_rate -= decay_factor

        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]

        for i in range(0, X.shape[0], batch_size):
            current_x = X[i:i+batch_size]
            current_y = y[i:i+batch_size]

            output, store = forward(current_x, W1, b1, W2, b2)
            dW1, db1, dW2, db2 = backward(current_x, current_y, store, W2)
            W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)

        output, _ = forward(X, W1, b1, W2, b2)
        global_loss, mean_loss = loss(y, output)
        print(f"Epoch {epoch+1}/{epochs}, Global Loss: {global_loss:.4f}, Mean Loss : {mean_loss}")

    return W1, b1, W2, b2

def predict(X, W1, b1, W2, b2):
    output, _ = forward(X, W1, b1, W2, b2)
    return np.argmax(output, axis=1)

X_train, y_train, X_test, y_test = load_data()
hidden_size = 7
n_classes = 10

W1, b1, W2, b2 = train(X_train, y_train, hidden_size, n_classes, learning_rate=0.2, epochs=10, batch_size=64, epoch_decay_interval=5)

y_pred = predict(X_test, W1, b1, W2, b2)
y_test = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

Epoch 1/10, Global Loss: 20632.4497, Mean Loss : 0.34387416169125556
Epoch 2/10, Global Loss: 27656.2076, Mean Loss : 0.46093679323616255
Epoch 3/10, Global Loss: 18811.3338, Mean Loss : 0.3135222294024702
Epoch 4/10, Global Loss: 17459.2583, Mean Loss : 0.2909876377065197
Epoch 5/10, Global Loss: 17540.7123, Mean Loss : 0.2923452048932402
Epoch 6/10, Global Loss: 17145.9591, Mean Loss : 0.2857659846937174
Epoch 7/10, Global Loss: 17955.3903, Mean Loss : 0.2992565042430903
Epoch 8/10, Global Loss: 17785.6295, Mean Loss : 0.29642715842418155
Epoch 9/10, Global Loss: 16516.7506, Mean Loss : 0.275279176364328
Epoch 10/10, Global Loss: 16921.9953, Mean Loss : 0.2820332556441828
Test Accuracy: 0.9139
